In [19]:
#Import the necessary packages
import pandas as pd
import numpy as np
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, HoverTool, Div, Range1d, LinearAxis, DataTable, \
TableColumn, RangeSlider, CustomJS, Select, CheckboxGroup,MultiSelect
from bokeh.layouts import row,column,gridplot
from bokeh.transform import factor_cmap,cumsum
from bokeh.palettes import Spectral6
from math import pi
from bokeh.models import ColumnDataSource
from bokeh.layouts import column, row

INTRODUCTION:

In this task, we will be analysing various datasets collected as a part of the NHANES program. We will be merging different kinds of datasets acquired from different types of examinations. Alot of data wrangling techniques which have been learnt throughout the course will be used to clean and pre-process data. We will be using the bokeh package to make different forms of visualisations to bring out various new insights.

The datasets that have been used for analysis throughout the task are Demographics data, blood pressure data, body measures, Cholesterol, blood pressure data, diabetes data, income data, medical conditions, mental health data and occupation data.

In [2]:
income_data = pd.read_sas('/Users/jasper/Downloads/P_INQ.XPT', format='xport')

In [3]:
demo_data = pd.read_sas('/Users/jasper/Downloads/P_DEMO.XPT', format='xport')

In [4]:
chol_data = pd.read_sas('/Users/jasper/Downloads/P_TCHOL.XPT', format='xport')

In [5]:
occ_data = pd.read_sas('/Users/jasper/Downloads/P_OCQ.XPT', format='xport')

In [6]:
med_data = pd.read_sas('/Users/jasper/Downloads/P_MCQ.XPT', format='xport')

In [7]:
diab_data = pd.read_sas('/Users/jasper/Downloads/P_DIQ.XPT', format='xport')

The depression data is just a question and answer response in form of scores. So, for this analysis, patient with a cummulative score of 5 or more are considered to be depressed.

In [8]:
depression_data = pd.read_sas('/Users/jasper/Downloads/P_DPQ.XPT', format='xport')
#The dataset contains 5.397605e-79 in place of 0, hence we replace them with the help of the below lambda function
depression_data = depression_data.apply(lambda x: np.where(np.isclose(x, 5.397605e-79), 0, x))
#Drop the unwanted column
depression_data = depression_data.drop("DPQ100",axis =1)
#Calculate the score
depression_data['score'] = depression_data.iloc[:, 1:].sum(axis=1)
counts = depression_data['score'].value_counts()
#Classify depressed or not and store it in a new column
depression_data['depression'] = depression_data['score'].apply(lambda x: 1 if x > 5 else 0)

In [9]:
hypert_data = pd.read_sas('/Users/jasper/Downloads/P_BPQ.XPT', format='xport')

In [10]:
bp_data = pd.read_sas('/Users/jasper/Downloads/P_BPXO.XPT', format='xport')

In [11]:
bmi_data = pd.read_sas('/Users/jasper/Downloads/P_BMX.XPT', format='xport')

In [20]:
merged_data_2021 = demo_data.merge(income_data, on='SEQN').merge(chol_data, on='SEQN').merge(depression_data, on='SEQN')\
.merge(occ_data, on='SEQN').merge(med_data, on='SEQN').merge(diab_data, on = 'SEQN').merge(hypert_data, on = 'SEQN')\
.merge(bp_data, on = 'SEQN').merge(bmi_data, on = 'SEQN')

To handle the missing values in columns like Cholesterol value, Systolic reading and diastolic reading, the medians of the corresponding columns have been used.

In [13]:
merged_data_2021['LBXTC'].fillna(merged_data_2021['LBXTC'].median(), inplace=True)
merged_data_2021['BPXOSY1'].fillna(merged_data_2021['BPXOSY1'].median(), inplace=True)
merged_data_2021['BPXODI1'].fillna(merged_data_2021['BPXODI1'].median(), inplace=True)

We will create a bar plot, that shows the count of people within each income level. Then on top of it, we will mark a line plot that shows the ratio of depressed people present in each bracket.

In [14]:
#Initialize a output notebook so that we can interact with the plots
output_notebook()

#INDFMMPC column contains the poverty index of each individual with 1 being on the lower end and 3 being on the higher end
# Count the number of people in each INDFMMPC category
indfmmpc_count = merged_data_2021['INDFMMPC'].value_counts().sort_index()

# Count the number of people with depression in each INDFMMPC category
depression_count = merged_data_2021.groupby('INDFMMPC')['depression'].sum().sort_index()

# Calculate the ratio of depressed people within the total people in that bracket
ratio = depression_count.values / indfmmpc_count.values

# Preparing the data for the plot
source = ColumnDataSource(data=dict(INDFMMPC=indfmmpc_count.index.astype(str), count=indfmmpc_count.values, depression_count=depression_count.values, ratio=ratio))

# Create the basic bar plot
p = figure(x_range=list(indfmmpc_count.index.astype(str)), title="Depression Count by INDFMMPC Category", \
           toolbar_location=None, tools="", height=400, width=600)

# Bar plot
p.vbar(x='INDFMMPC', top='count', width=0.9, source=source, legend_field="INDFMMPC", line_color='white', \
       fill_color=factor_cmap('INDFMMPC', palette=Spectral6, factors=list(indfmmpc_count.index.astype(str))))

# Add a secondary y-axis for the ratio plot
p.extra_y_ranges = {"ratio": Range1d(start=0, end=max(ratio) * 1.1)}
p.add_layout(LinearAxis(y_range_name="ratio", axis_label='Depression Ratio'), 'right')

# Line plot for the ratio on the secondary y-axis
p.line(x='INDFMMPC', y='ratio', source=source, line_width=2, color='red', legend_label="Depression Ratio", y_range_name="ratio")
p.circle(x='INDFMMPC', y='ratio', source=source, size=8, color='red', legend_label="Depression Ratio", y_range_name="ratio")

p.xgrid.grid_line_color = None
p.y_range.start = 0
p.yaxis[0].axis_label = 'Number of People'
p.xaxis.axis_label = 'INDFMMPC Category'
p.legend.orientation = "horizontal"
p.legend.location = "top_center"

# Create a HoverTool to display information when hovering over the bars and the line
hover = HoverTool(tooltips=[
    ("INDFMMPC Category", "@INDFMMPC"),
    ("Count", "@count"),
    ("Depression Count", "@depression_count"),
    ("Ratio", "@ratio")
])
p.add_tools(hover)

# Create a Div element to display additional instructions
div = Div(text="Hover over the bars and points to see details.", width=600)

# Layout both the plots
layout = column(p, div)

# Show the layout
show(layout)


Loading BokehJS ...

Instructions:
As you hover the bar or the circles on the line, you will be able to view the detailed values corresponding to that particular bar or line.

From the above plot, we could observe that, as the income gets higher, the ratio of depressed people is decreasing. (Labels 7 are missing data and 9 are people who didn't disclose the data). Hence, we can infer that a person's income is one of the key factors to maintain his mental health. 

We will create a line plot with some alterations, to view mean choleserol value for each age group.

In [15]:
#Plot mean cholesterol for each age
nhanes_data_chol = merged_data_2021[['RIDAGEYR', 'LBXTC']]

# Remove missing values if any
nhanes_data_chol = nhanes_data_chol.dropna()

# Group by age and calculate the mean cholesterol for each age group
mean_cholesterol_by_age = nhanes_data_chol.groupby('RIDAGEYR')['LBXTC'].mean().reset_index()

# Create a ColumnDataSource for the plot
source = ColumnDataSource(data=mean_cholesterol_by_age)

# Create a scatter plot of Mean Cholesterol against Age
p = figure(title="Mean Cholesterol Levels vs. Age", x_axis_label='Age', y_axis_label='Mean Cholesterol')

# Add circle glyphs for each data point
p.line(x='RIDAGEYR', y='LBXTC', source=source, line_dash="4 4", line_width=1, color='gray')
#p.circle(x='RIDAGEYR', y='LBXTC', source=source, size=8, color='blue', alpha=0.7)
cr = p.circle(x='RIDAGEYR', y='LBXTC', source=source, size=20,
              fill_color="grey", hover_fill_color="firebrick",
              fill_alpha=0.05, hover_alpha=0.3,
              line_color=None, hover_line_color="white")
# Add hover tooltips to display additional information on hover
#hover = HoverTool(tooltips=[("Age", "@RIDAGEYR"), ("Mean Cholesterol", "@LBXTC")])
#p.add_tools(hover)
p.add_tools(HoverTool(tooltips=[("Age", "@RIDAGEYR"), ("Mean Cholesterol", "@LBXTC")], renderers=[cr], mode='hline'))
# Show the plot
output_notebook()
show(p)


Loading BokehJS ...

Instructions:
As you hover over the screen , you will be able to view a tooltip box containing the details of age group with almost equal mean cholesterol level

From the above plot we can draw some inferences such as,
1)As age increases cholesterol level also increases gradually till a particular age (till 60s)
2)The cholesterol gradually decreases
3)Aged people should be careful about their diet in terms of cholesterol for health reasons

In the next plot, we will be making plots to see if working hours has an impact on health conditions such as Cholesterol, hyper-tension, diabetes, Asthma, heart attack, liver conditions and thyroid. We will be doing this with the help of linked plots from the bokeh package.

In [16]:
output_notebook()
df = merged_data_2021

# Define the health conditions 
conditions = {
    'Cholesterol': df['BPQ080'] == 1,
    'Hyper-tension' : df['BPQ020'] == 1,
    'Diabetes': df['DIQ010'] == 1,
    'Asthma': df['MCQ010'] == 1,
    'Heart Attack': df['MCQ160E'] == 1,
    'Thyroid': df['MCQ160M'] == 1,
    'Liver Condition': df['MCQ160L'] == 1
}

# Define the work-hours mapping
occupation_mapping = {
    1: '9-5',
    2: 'night',
    3: 'early morning',
    5: 'varying'
}

# Create the counts for each condition for varying work hours
occupation_data = { 'occupation': ['9-5', 'night', 'early morning', 'varying'] }
for condition, mask in conditions.items():
    occupation_data[condition] = [
        df[mask & (df['OCQ670'] == occ)].shape[0] for occ in occupation_mapping.keys()
    ]

# Convert health condition data to DataFrame so that it can be used for plotting
df_health = pd.DataFrame({
    'condition': list(conditions.keys()),
    'count': [df[mask].shape[0] for mask in conditions.values()]
})

df_occupation = pd.DataFrame(occupation_data)

# Initial bar plot data source
source_bar = ColumnDataSource(df_health)

# Initial pie chart data source
initial_condition = df_health['condition'][0]
pie_data = {
    'occupation': df_occupation['occupation'],
    'values': df_occupation[initial_condition]
}
#calculating angle for each entry in pie-chart
pie_data['angle'] = pie_data['values'] / pie_data['values'].sum() * 2 * pi
pie_data['color'] = ['#f25c5c', '#5c9ef2', '#f2e25c', '#5cf285']
source_pie = ColumnDataSource(pie_data)

# Create bar plot
p1 = figure(x_range=df_health['condition'], title="Count of Health Conditions", height=500, width=500, tools="tap")
p1.vbar(x='condition', top='count', width=0.9, source=source_bar, color="navy", alpha=0.5)

# Create pie chart
p2 = figure(height=500, width=500, title="Occupation Distribution", toolbar_location=None, tools="hover",\
            tooltips="@occupation: @values", x_range=(-0.5, 1.0))
p2.wedge(x=0, y=1, radius=0.4, start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),\
         line_color="white", fill_color='color', legend_field='occupation', source=source_pie)
p2.axis.axis_label = None
p2.axis.visible = False
p2.grid.grid_line_color = None

# Updation of pie chart when bar is clicked using javascript callback
source_bar.selected.js_on_change(
    'indices',
    CustomJS(args=dict(source_bar=source_bar, source_pie=source_pie, df_occupation=df_occupation.to_dict('list')), code="""
    var indices = cb_obj.indices;
    var data_bar = source_bar.data;
    var data_pie = source_pie.data;
    var occupation_data = df_occupation;

    if (indices.length == 0) {
        return;
    }

    var index = indices[0];
    var condition = data_bar['condition'][index];

    // Update pie chart values based on selected condition from bar graph
    data_pie['values'] = occupation_data[condition];
    var total = data_pie['values'].reduce((a, b) => a + b, 0);

    for (var i = 0; i < data_pie['values'].length; i++) {
        data_pie['angle'][i] = data_pie['values'][i] / total * 2 * Math.PI;
    }

    source_pie.change.emit();
    """)
)

layout = row(p1, p2)

show(layout)

Loading BokehJS ...

Instructions:
1)Once you run the cell a tool box along with a tap icon is placed next to the bar graph. Make sure that you select it (a blue line on the left edge means selected)
2)Now click on one of the bars in the graph. The corresponding pie graph will emerge on the other side.
3)To select another bar, click on the tap icon once again to deselect, and click on it again to select the tap functionality.
4)Now click on the other desired bar.

The above plot is helpful in viewing the number of people working in varying shifts who suffer from a particular health condition. We can observe that, most of the health conditions are found in people working either the regular shift or varying shift. This might be due to the fact that the people working in those shifts are more. But if we take out the ratio, we might be able to make other inferences as well.

In the next plot, we will be plotting the Systole and Diastole readings against the age to view the pattern. This is also done by linking 2 plots. As the range in 1 plot is changed by zooming in or moving sideways, the next plot will also show the corresponding co-ordinates.

In [17]:
output_notebook()

# Aggregate data by age and calculate mean values
mean_data = merged_data_2021.groupby('RIDAGEYR').agg({
    'BPXOSY1': 'mean',
    'BPXODI1': 'mean'
}).reset_index()

# Create a ColumnDataSource
source = ColumnDataSource(mean_data)

# Create the first line plot (Mean Systolic Blood Pressure)
p1 = figure(title="Mean Systolic Blood Pressure vs Age", width=400, height=400)
p1.circle('RIDAGEYR', 'BPXOSY1', size=10, source=source, color="navy", alpha=0.5)
p1.line('RIDAGEYR', 'BPXOSY1', source=source, color="navy", alpha=0.5)

# Create the second line plot (Mean BMI)
p2 = figure(title="Mean Diastolic Blood Pressure vs Age", width=400, height=400, x_range=p1.x_range)
p2.circle('RIDAGEYR', 'BPXODI1', size=10, source=source, color="firebrick", alpha=0.5)
p2.line('RIDAGEYR', 'BPXODI1', source=source, color="firebrick", alpha=0.5)

# Arrange the plots in a grid
layout = gridplot([[p1, p2]])

# Show the layout
show(layout)


Loading BokehJS ...

INSTRUCTIONS:
There is a tool box here as well. The first icon when selected enables you to move throughout the graph and explore. The second option when selected enables you to select a section of one of the graphs and view the same section on the other graph. The third icon when selected, enables you to zoom in and out in the graph.

From the above graph we can infer that the systolic blood pressure increases gradually with age within a range of 110 till 150. This clears the notion that 120 is the ideal systolic pressure. The mean diastolic pressure increases till late 50s and starts to decrease gradually. The range is between 67 - 80

We will create a new table with various functionalities such as sorting based on a column, selecting from drop down, checkboxes, slider for ranges to get a comprehensive look on how the records are present.

In [18]:
output_notebook()

# data needed to create the table
data = {
    'Age': merged_data_2021['RIDAGEYR'],
    'Gender': merged_data_2021['RIAGENDR'],  
    'BMI': merged_data_2021['BMXBMI'],
    'Systolic_BP': merged_data_2021['BPXOSY1'],
    'Diastolic_BP': merged_data_2021['BPXODI1'],
    'Cholesterol': merged_data_2021['BPQ080'] == 1,
    'Hyper-tension': df['BPQ020'] == 1,
    'Diabetes': df['DIQ010'] == 1,
    'Asthma': df['MCQ010'] == 1,
    'Depressed': merged_data_2021['depression'],
    'Heart_Condition': df['MCQ160E'] == 1,
    'Thyroid': df['MCQ160M'] == 1,
    'Liver_Condition': df['MCQ160L'] == 1,
    'Work_Time': df['OCQ670']
}

nhanes_df = pd.DataFrame(data)

# Create a ColumnDataSource
source = ColumnDataSource(nhanes_df)
filtered_source = ColumnDataSource(nhanes_df)

# Define the columns for the table
columns = [
    TableColumn(field='Age', title='Age'),
    TableColumn(field='Gender', title='Gender'),
    TableColumn(field='BMI', title='BMI'),
    TableColumn(field='Systolic_BP', title='Systolic Blood Pressure'),
    TableColumn(field='Diastolic_BP', title='Diastolic Blood Pressure'),
    TableColumn(field='Cholesterol', title='Cholesterol'),
    TableColumn(field='Depressed', title='Depressed'),
    TableColumn(field='Heart_Condition', title='Heart Condition'),
    TableColumn(field='Liver_Condition', title='Liver Condition'),
]

# Create the DataTable
data_table = DataTable(source=filtered_source, columns=columns, width=800, height=280)

# Create the necessary widgets to create the filters
age_slider = RangeSlider(start=nhanes_df['Age'].min(), end=nhanes_df['Age'].max(), value=(nhanes_df['Age'].min(),
                            nhanes_df['Age'].max()), step=1, title="Age Range")
gender_select = Select(title="Gender", value="All", options=["All", "Male", "Female"])
work_time_select = Select(title="Work Time", value="All", options=["All", "Traditional 9 AM to 5 PM day", 
                         "Evening or nights", "Early mornings", "Variable (early mornings, days, and nights)"])
checkboxes = CheckboxGroup(labels=["Depressed", "Heart Condition", "Liver Condition"], active=[])

# CustomJS callback to filter data based on widget values
callback = CustomJS(args=dict(source=source, filtered_source=filtered_source, age_slider=age_slider, 
                              gender_select=gender_select, work_time_select=work_time_select, checkboxes=checkboxes), code="""
    const data = source.data;
    const filtered_data = { Age: [], Gender: [], BMI: [], Systolic_BP: [], Diastolic_BP: [], 
    Cholesterol: [], Depressed: [], Heart_Condition: [], Liver_Condition: [], Work_Time: [] };

    const age_range = age_slider.value;
    const gender = gender_select.value;
    const work_time = work_time_select.value;
    const active_checks = checkboxes.active.map(i => checkboxes.labels[i]);

    const work_time_map = {
        1: "Traditional 9 AM to 5 PM day",
        2: "Evening or nights",
        3: "Early mornings",
        5: "Variable (early mornings, days, and nights)"
    };

    for (let i = 0; i < data['Age'].length; i++) {
        const is_depressed = data['Depressed'][i];
        const has_heart_condition = data['Heart_Condition'][i];
        const has_liver_condition = data['Liver_Condition'][i];
        const work_time_label = work_time_map[data['Work_Time'][i]];
        const matches_work_time = work_time === "All" || work_time_label === work_time;
        
        // Convert numeric gender to label for comparison
        const gender_label = data['Gender'][i] === 1.00 ? "Male" : "Female";

        if ((gender === "All" || gender_label === gender) &&
            (data['Age'][i] >= age_range[0] && data['Age'][i] <= age_range[1]) &&
            matches_work_time &&
            (!active_checks.includes("Depressed") || is_depressed) &&
            (!active_checks.includes("Heart Condition") || has_heart_condition) &&
            (!active_checks.includes("Liver Condition") || has_liver_condition)) {
            filtered_data['Age'].push(data['Age'][i]);
            filtered_data['Gender'].push(gender_label);  // Push the label, not the numeric value
            filtered_data['BMI'].push(data['BMI'][i]);
            filtered_data['Systolic_BP'].push(data['Systolic_BP'][i]);
            filtered_data['Diastolic_BP'].push(data['Diastolic_BP'][i]);
            filtered_data['Cholesterol'].push(data['Cholesterol'][i]);
            filtered_data['Depressed'].push(data['Depressed'][i]);
            filtered_data['Heart_Condition'].push(data['Heart_Condition'][i]);
            filtered_data['Liver_Condition'].push(data['Liver_Condition'][i]);
            filtered_data['Work_Time'].push(work_time_label);
        }
    }

    filtered_source.data = filtered_data;
    filtered_source.change.emit();
""")

age_slider.js_on_change('value', callback)
gender_select.js_on_change('value', callback)
work_time_select.js_on_change('value', callback)
checkboxes.js_on_change('active', callback)

# Layout the widgets and the table
layout = column(row(age_slider, gender_select, work_time_select), row(checkboxes), data_table)

# Show the layout
show(layout)


Loading BokehJS ...

From the above table, various filters can be applied and the dataset can be filtered out to find various subsets of data for further analysis.

CONCLUSION:
Throughout this task, we have drawn various conclusions based on the plots which we have plotted by combining a variety of datasets. The bokeh package has been extensively used to make interactive plots, linked plots and table. Effective visualization techniques have been used to bring out conclusions and inferences between various variables from the NHANES dataset.